In [104]:
import turicreate as tc
import numpy as np

In [105]:
tc.__version__

'6.4.1'

In [106]:
from pymongo import MongoClient

In [107]:
client = MongoClient(serverSelectionTimeoutMS=50)
db = client.turidatabase

In [108]:
db.labeledinstances.find({'dsid': 5})

In [109]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=50), 'turidatabase')

In [110]:
def get_dataset_data(dsid):
    features = []
    labels = []
    
    for a in db.labeledinstances.find({"dsid": dsid}):
        features.append([float(val) for val in a['feature']])
        labels.append(a['label'])
    data = {'target': labels, 'sequence':np.array(features)}
    
    return data


def get_dataset_sframe(dsid):
    data = get_dataset_data(dsid)
    return tc.SFrame(data=data)


In [111]:
sframe_colors = tc.SFrame.read_csv('./capture-mathod-flash-on.csv')

Finished parsing file /Users/nicholaslarsen/Documents/7323 Projects/LearnedRubiks/Notebooks/capture-mathod-flash-on.csv

Parsing completed. Parsed 100 lines in 0.032362 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,float,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/nicholaslarsen/Documents/7323 Projects/LearnedRubiks/Notebooks/capture-mathod-flash-on.csv

Parsing completed. Parsed 406 lines in 0.004015 secs.

In [112]:
sframe_colors

red,green,blue,target
226.24744897959184,202.7944606413994,181.40962099125363,white
160.03279883381924,9.325801749271136,12.466107871720116,red
220.64395043731776,199.8338192419825,176.76384839650146,white
241.21137026239063,103.66180758017492,29.419460641399414,orange
71.08491253644314,151.94460641399417,60.61807580174927,green
196.34365889212827,179.62208454810497,162.02004373177843,white
195.65524781341108,181.6865889212828,166.10204081632654,white
197.48943148688048,70.49781341107871,6.662900874635568,orange
208.2084548104956,91.54227405247812,18.651967930029155,orange
160.70845481049562,22.25546647230321,20.51639941690962,red


In [113]:
 import turicreate.aggregate as agg
sframe_colors.groupby('target',operations={'count': agg.COUNT()})

target,count
green,66
orange,71
blue,65
yellow,70
red,69
white,65


## Create a defualt model

In [203]:
sframe_colors = tc.SFrame.read_csv('./capture-mathod-flash-on.csv')
data_train, data_test = sframe_colors.random_split(0.8)
model = tc.classifier.create(data_train, target='target', verbose=False)
yhat = model.predict(data_test)
acc = sum(yhat == data_test['target'])/ float(len(data_test))
print('Acc:', acc)
model.export_coreml('./colors.mlmodel')

Finished parsing file /Users/nicholaslarsen/Documents/7323 Projects/LearnedRubiks/Notebooks/capture-mathod-flash-on.csv

Parsing completed. Parsed 100 lines in 0.047579 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,float,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/nicholaslarsen/Documents/7323 Projects/LearnedRubiks/Notebooks/capture-mathod-flash-on.csv

Parsing completed. Parsed 694 lines in 0.004669 secs.

Acc: 0.9727891156462585


In [205]:
data_test['yhat'] = yhat 

In [116]:
x = data_test['target' == 'red']

In [117]:
sf_filter = data_test[data_test.apply(lambda x: x['yhat'] != x['target'])] 

In [206]:
data_test[data_test['target'] != data_test['yhat']]

red,green,blue,target,yhat
137.71865889212827,136.42747813411077,21.60131195335277,yellow,red
208.1793002915452,189.97995626822157,28.68258017492711,yellow,orange
172.8895772594752,68.11953352769679,15.129737609329446,orange,red
201.74125364431487,182.951166180758,25.20298833819242,yellow,orange


In [119]:
sf_filter

red,green,blue,target,yhat
46.464650145772595,107.39686588921283,37.91909620991254,green,blue
148.5783527696793,136.3043002915452,122.82179300291546,white,blue
